In [ ]:
import nltk
import numpy as np
import pandas as pd
import re

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report, f1_score, recall_score, accuracy_score, precision_score

from imblearn.over_sampling import RandomOverSampler

nltk.download("punkt")
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

df = pd.read_csv("/content/drive/MyDrive/DeGatto Project/EDA + Models/Womens Clothing E-Commerce Reviews.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df.drop(["Unnamed: 0", "Clothing ID","Age", "Title", "Positive Feedback Count", "Division Name", "Department Name", "Class Name"], axis = 1)
df.dropna(inplace = True)

def categorise(row):  
    if row['Rating'] >= 4 and row['Recommended IND'] == 1:
        return 'Positive'
    elif row['Rating'] >= 4 and row['Recommended IND'] == 0:
        return 'Neutral'
    elif row['Rating'] == 3:
        return 'Neutral'
    elif row['Rating'] <= 2 and row['Recommended IND'] == 1:
        return 'Neutral'
    elif row['Rating'] <= 2 and row['Recommended IND'] == 0:
        return 'Negative'

df['Sentiment'] = df.apply(lambda row: categorise(row), axis = 1)

df.drop(['Rating', 'Recommended IND'], axis = 1, inplace = True)

reviews = df['Review Text'].str.strip()

NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')

def normalize_texts(texts):
    normalized_texts = []

    for text in texts:

        ## Lower Casing
        lower = str(text).lower()

        ## Removing Punctuations
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)

        ## Removing NON_ASCII values
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)

        ## Removing numbers
        no_num = re.sub('\W+','', no_non_ascii )

        ## appending to the list
        normalized_texts.append(no_non_ascii)

    return normalized_texts
        
df['Review Text'] = normalize_texts(df['Review Text'])

## Replacing the multiple whitespaces with just one
df["Review Text"] = df["Review Text"].replace(r'\s+', ' ', regex=True)

In [ ]:
x = df["Review Text"]
y = df["Sentiment"]

vectorizer = CountVectorizer( max_features = 5098)
over = RandomOverSampler(sampling_strategy = 'all', random_state = 45)
x = vectorizer.fit_transform(x)
x, y = over.fit_resample(x, y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, 
                                                    stratify = y, random_state = 42)

log = LogisticRegression(C = 2, max_iter = 500, class_weight = 'balanced',
                         random_state = 42)

log.fit(x_train, y_train)
y_pred_test = log.predict(x_test)
y_pred_train = log.predict(x_train)

acc_test = accuracy_score(y_test, y_pred_test)
acc_train = accuracy_score(y_train, y_pred_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import joblib

In [ ]:
df

,Review Text,Rating,Recommended IND
0,absolutely wonderful silky and sexy and comfor...,4,1
1,love this dress it s sooo pretty i happened to...,5,1
2,i had such high hopes for this dress and reall...,3,0
3,i love love love this jumpsuit it s fun flirty...,5,1
4,this shirt is very flattering to all due to th...,5,1
...,...,...,...
23481,i was very happy to snag this dress at such a ...,5,1
23482,it reminds me of maternity clothes soft stretc...,3,1
23483,this fit well but the top was very see through...,3,0
23484,i bought this dress for a wedding i have this ...,3,1


In [ ]:
filename = '/content/drive/My Drive/DeGatto Project/EDA + Models/finalized_model.sav'
from google.colab import drive
drive.mount('/content/drive') 
df = pd.read_csv("/content/drive/MyDrive/DeGatto Project/EDA + Models/Womens Clothing E-Commerce Reviews.csv")
loaded_model = joblib.load(filename)
df = df.drop(["Unnamed: 0", "Clothing ID","Age", "Title", "Positive Feedback Count", "Division Name", "Department Name", "Class Name"], axis = 1)
df.dropna(inplace = True)
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []

    for text in texts:

        ## Lower Casing
        lower = str(text).lower()

        ## Removing Punctuations
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)

        ## Removing NON_ASCII values
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)

        ## Removing numbers
        no_num = re.sub('\W+','', no_non_ascii )

        ## appending to the list
        normalized_texts.append(no_non_ascii)

    return normalized_texts
df['Review Text'] = normalize_texts(df['Review Text'])
df["Review Text"] = df["Review Text"].replace(r'\s+', ' ', regex=True)
x = df["Review Text"]
y = df["Rating"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, 
                                                    stratify = y, random_state = 42)
vectorizer = CountVectorizer(max_features = 5098)
x = vectorizer.fit_transform(x_test)
loaded_model.predict(x)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ValueError: ignored